### Text to speech in Romanian
Reference: https://github.com/anurag265/GPT-Examples/blob/main/PDF%20to%20Audiobook/PDF%20to%20Audiobook.ipynb

Limitation: OpenAI's text to speech considers only 4096 characters.
If provided text is longer, it must be chunked.

**this is bold** and this is not. _Italic_

### Split text into chunks
The following function splits the provided text into chunks. Maximum chunk size defaults to 4096

In [2]:
def split_text(text, max_chunk_size=4096):
    chunks = []  # List to hold the chunks of text
    current_chunk = ""  # String to build the current chunk

    # Split the text into sentences and iterate through them
    for sentence in text.split('.'):
        sentence = sentence.strip()  # Remove leading/trailing whitespaces
        if not sentence:
            continue  # Skip empty sentences

        # Check if adding the sentence would exceed the max chunk size
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += sentence + "."  # Add sentence to current chunk
        else:
            chunks.append(current_chunk)  # Add the current chunk to the list
            current_chunk = sentence + "."  # Start a new chunk

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

### Text-to-Speech Conversion


In [5]:
# Importing necessary modules
from pathlib import Path
import openai

def text_to_speech(input_text, output_file, model="tts-1-hd", voice="nova"):
    # Initialize the OpenAI client
    client = openai.OpenAI()

    # Make a request to OpenAI's Audio API with the given text, model, and voice
    response = client.audio.speech.create(
        model=model,      # Model for text-to-speech quality
        voice=voice,      # Voice type
        input=input_text  # The text to be converted into speech
    )

    # Define the path for the output audio file
    speech_file_path = Path(output_file)

    # Stream the audio response to the specified file
    response.stream_to_file(speech_file_path)

    # Print confirmation message after saving the audio file
    print(f"Audio saved to {speech_file_path}")

Convert chunks to audio. Each chunk has its own audio file.

In [8]:
# Importing necessary modules
import os

def convert_chunks_to_audio(chunks, output_folder):
    audio_files = []  # List to store the paths of generated audio files

    # Iterate over each chunk of text
    for i, chunk in enumerate(chunks):
        # Define the path for the output audio file
        output_file = os.path.join(output_folder, f"chunk_{i+1}.mp3")

        # Convert the text chunk to speech and save as an audio file
        text_to_speech(chunk, output_file, model='tts-1-hd', voice='onyx')

        # Append the path of the created audio file to the list
        audio_files.append(output_file)

    return audio_files  # Return the list of audio file paths

In [9]:
def read_content(filename):
    with open(filename, 'r') as f:
        return f.read()
    
content = read_content('text/text2.txt')

chunks = split_text(content)

convert_chunks_to_audio(chunks, 'audio_temp')

Audio saved to audio_temp/chunk_1.mp3
Audio saved to audio_temp/chunk_2.mp3
Audio saved to audio_temp/chunk_3.mp3
Audio saved to audio_temp/chunk_4.mp3


['audio_temp/chunk_1.mp3',
 'audio_temp/chunk_2.mp3',
 'audio_temp/chunk_3.mp3',
 'audio_temp/chunk_4.mp3']

### Merge the chunks

Merge the mp3 chunks into a single mp3 file.

In [7]:
import re
from moviepy.editor import concatenate_audioclips, AudioFileClip
import os

def extract_number(filename):
    """ Extracts the number from the filename """
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

def combine_audio_with_moviepy(folder_path, output_file):
    audio_clips = []  # List to store the audio clips

    # Retrieve and sort files based on the numeric part of the filename
    sorted_files = sorted(os.listdir(folder_path), key=extract_number)

    # Iterate through each sorted file in the given folder
    for file_name in sorted_files:
        if file_name.endswith('.mp3'):
            # Construct the full path of the audio file
            file_path = os.path.join(folder_path, file_name)
            print(f"Processing file: {file_path}")

            try:
                # Create an AudioFileClip object for each audio file
                clip = AudioFileClip(file_path)
                audio_clips.append(clip)  # Add the clip to the list
            except Exception as e:
                # Print any errors encountered while processing the file
                print(f"Error processing file {file_path}: {e}")

    # Check if there are any audio clips to combine
    if audio_clips:
        # Concatenate all the audio clips into a single clip
        final_clip = concatenate_audioclips(audio_clips)
        # Write the combined clip to the specified output file
        final_clip.write_audiofile(output_file)
        print(f"Combined audio saved to {output_file}")
    else:
        print("No audio clips to combine.")


# Function Usage
combine_audio_with_moviepy('audio_temp', 'combined_audio.mp3')

Processing file: audio_temp/chunk_1.mp3
Processing file: audio_temp/chunk_2.mp3
Processing file: audio_temp/chunk_3.mp3
Processing file: audio_temp/chunk_4.mp3
MoviePy - Writing audio in combined_audio.mp3


MoviePy - Done.
Combined audio saved to combined_audio.mp3
